In [1]:
import os
import glob
import tensorflow as tf
import numpy as np
import dask
import dask.array as da
from PIL import Image
from skimage.transform import resize
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import scipy
%matplotlib inline
np.set_printoptions(threshold=10000, linewidth=32)

In [2]:
y = np.asarray(sorted(glob.glob("/home/skyolia/JupyterProjects/data/BASE-CYTO/images etiquetées/*.bmp")))
x = np.asarray(sorted(glob.glob("/home/skyolia/JupyterProjects/data/BASE-CYTO/images originales/*.bmp")))

In [3]:
train = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,9,
 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
 30, 31, 32, 33, 34, 35, 53, 54, 55, 56,
 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
 77, 78, 79, 80, 81, 82, 83, 84, 85, 86]
test = [36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
 46, 47, 48, 49, 50, 51, 52]
#train_x, train_y = x[train], y[train]
#test_x, test_y = x[test], y[test]
#train_x, train_y, test_x, test_y

In [3]:
rnd = np.random.choice(87, 61, False)
#train_x, train_y = x[train_idx], y[train_idx]

splitter = np.isin(np.arange(87), rnd, True)
train, test = np.where(splitter)[0], np.where(~splitter)[0]

np.sum(splitter), train, test

(61, array([ 0,  1,  4,  5,  6,  7,
         8,  9, 11, 12, 16, 17,
        20, 22, 23, 24, 26, 27,
        29, 31, 32, 34, 35, 38,
        40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 52, 53,
        54, 55, 56, 58, 61, 63,
        64, 66, 67, 69, 70, 72,
        73, 74, 75, 76, 77, 78,
        79, 80, 81, 82, 83, 85,
        86]), array([ 2,  3, 10, 13, 14, 15,
        18, 19, 21, 25, 28, 30,
        33, 36, 37, 39, 50, 51,
        57, 59, 60, 62, 65, 68,
        71, 84]))

In [4]:
def padding(obj, i):
    pad_size = np.asarray(obj) - np.asarray(i.shape)
    padded = np.pad(i, ((0, pad_size[0]), (0, pad_size[1]), (0, pad_size[2])), mode='edge')
    return padded

def extract_patch_single_image(i, crop_size):
    crops = []
    height, width = i.shape[0], i.shape[1]
    for h in range(0, height, 25):
        for w in range(0, width, 25):
            image = i[h:h+crop_size[0], w:w+crop_size[1], :]
            image = padding((crop_size[0], crop_size[1], image.shape[2]), image) if image.shape[0:2] != crop_size else image
            crops.append(image)
    return np.asarray(crops)

def load_save(path, mode, crop_size, new_shape=None):
    #image = Image.open(path).convert(mode)
    image = scipy.ndimage.imread(path, mode=mode)
    image = np.asarray(image)#/255.0
    image = image[:, :, np.newaxis] if image.ndim == 2 else image
    image = extract_patch_single_image(image, crop_size) if crop_size else image
    image = resize(image, new_shape, preserve_range = True) if new_shape else image
    return image#.astype(int)

def data_processing(data_paths, crop_size, new_shape, mode, chunk, stack=True):
    x = []
    for i in data_paths:
        image = dask.delayed(load_save)(i, mode, crop_size, new_shape=None)
        x.append(da.from_delayed(image, new_shape, dtype=int))
    res = da.stack(x, axis=0).rechunk(chunk) if stack else da.concatenate(x, axis=0).rechunk(chunk)
    return res

train_x = data_processing(x[train], (32, 32), (1271, 32, 32, 3), 'RGB', (15000, 32, 32, 3), False)
train_y = data_processing(y[train], (32, 32), (1271, 32, 32, 1), 'L', (15000, 32, 32, 1), False)
#colored_train_y = data_processing(masks[train], (32, 32), (7400, 32, 32, 3), 'RGB', (10000, 32, 32, 3), False)
test_x = data_processing(x[test], None, (768, 1024, 3), 'RGB', (1, 768, 1024, 3))
test_y = data_processing(y[test], None, (768, 1024, 1), 'L', (1, 768, 1024, 1))
#colored_test_y = data_processing(masks[test], None, (768, 1024, 3), 'RGB', (1, 768, 1024, 3))
train_x, train_y, test_x, test_y

(dask.array<rechunk-merge, shape=(88970, 32, 32, 3), dtype=int64, chunksize=(15000, 32, 32, 3)>,
 dask.array<rechunk-merge, shape=(88970, 32, 32, 1), dtype=int64, chunksize=(15000, 32, 32, 1)>,
 dask.array<stack, shape=(17, 768, 1024, 3), dtype=int64, chunksize=(1, 768, 1024, 3)>,
 dask.array<stack, shape=(17, 768, 1024, 1), dtype=int64, chunksize=(1, 768, 1024, 1)>)

In [6]:
#train_y = da.where(train_y==118, 117, train_y)
%time da.unique(train_y).compute(), da.unique(test_y).compute()

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


CPU times: user 3.32 s, sys: 2.12 s, total: 5.44 s
Wall time: 9.07 s


(array([  0,  59, 117, 177],
       dtype=uint8), array([  0,  59, 117, 177],
       dtype=uint8))

In [5]:
def one_hot_labels_convertion(y):
    res = []
    for i in da.unique(y).compute():
        res.append(da.where(y==i, 1., 0))
    return da.concatenate(res, axis=3)

train_y = one_hot_labels_convertion(train_y)
test_y = one_hot_labels_convertion(test_y)
train_x, test_x = train_x/255., test_x/255.
train_x, train_y, test_x, test_y

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


(dask.array<truediv, shape=(88970, 32, 32, 3), dtype=float64, chunksize=(15000, 32, 32, 3)>,
 dask.array<concatenate, shape=(88970, 32, 32, 4), dtype=float64, chunksize=(15000, 32, 32, 1)>,
 dask.array<truediv, shape=(17, 768, 1024, 3), dtype=float64, chunksize=(1, 768, 1024, 3)>,
 dask.array<concatenate, shape=(17, 768, 1024, 4), dtype=float64, chunksize=(1, 768, 1024, 1)>)

In [6]:
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters, dropout=0.1):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters) #(32, 32, 16)
    conv_2 = build_block(conv_1, n_filters) #(32, 32, 16)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)#(16, 16, 16)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)#(16, 16, 32)
    conv_4 = build_block(conv_3, n_filters * 2)#(16, 16, 32)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)#(8, 8, 32)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)#(8, 8, 64)
    conv_6 = build_block(conv_5, n_filters * 4) #(8, 8, 64)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)#(4, 4, 64)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)#(4, 4, 128)
    conv_8 = build_block(conv_7, n_filters * 8) #(4, 4, 128)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_8) #(-1, 8, 8, 128)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_6]) #(-1, 8, 8, 192)
    conv_9 = build_block(upsp_1, n_filters * 4) #(-1, 8, 8, 64)
    conv_10 = build_block(conv_9, n_filters * 4)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_10)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 16, 16, 128)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 16, 16, 192)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 16, 16, 64)
    conv_16 = build_block(conv_15, n_filters * 2)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_16)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 32, 32, 64)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 32, 32, 92)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 32, 32, 32)
    conv_18 = build_block(conv_17, n_filters)
    drop_8 = tf.keras.layers.SpatialDropout2D(dropout)(conv_18)
    
    output = tf.keras.layers.Conv2D(4, (1, 1), kernel_initializer='glorot_normal', activation='softmax')(drop_8)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model
        
model = build_unet(input_shape=(None, None, 3), n_filters=16)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 432         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 1 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [7]:
model.load_weights("day_1.weights.best.hdf5")
opt = tf.keras.optimizers.Adam() # 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
scores = model.evaluate(test_x, test_y, batch_size=1)
scores

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


17/17 [==============================] - 7s 422ms/step


[0.2183622774832389, 0.9225624729605282]

In [8]:
epochs, batch_size, lr, filepath = 1000000, 256, 0.001, "day_1.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(horizontal_flip=True, vertical_flip=True)#, width_shift_range=0.1, height_shift_range=0.1
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.metrics_names)

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    initial_epoch=0,
                    callbacks=[checkpoint, tb])
#tensorboard --logdir=/home/skyolia/JupyterProjects/segmentation/BASE-CYTO/

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


['loss', 'acc']
Epoch 1/1000000
347/348 [============================>.] - ETA: 0s - loss: 74.0348 - acc: 0.9265
Epoch 00001: val_acc improved from -inf to 0.83733, saving model to day_1.weights.best.hdf5
348/348 [==============================] - 132s 378ms/step - loss: 73.8912 - acc: 0.9266 - val_loss: 0.4674 - val_acc: 0.8373
Epoch 2/1000000
347/348 [============================>.] - ETA: 0s - loss: 46.7933 - acc: 0.9419
Epoch 00002: val_acc improved from 0.83733 to 0.86841, saving model to day_1.weights.best.hdf5
348/348 [==============================] - 55s 157ms/step - loss: 46.7065 - acc: 0.9420 - val_loss: 0.4057 - val_acc: 0.8684
Epoch 3/1000000
347/348 [============================>.] - ETA: 0s - loss: 40.1355 - acc: 0.9480
Epoch 00003: val_acc improved from 0.86841 to 0.91337, saving model to day_1.weights.best.hdf5
348/348 [==============================] - 56s 161ms/step - loss: 40.0763 - acc: 0.9480 - val_loss: 0.2553 - val_acc: 0.9134
Epoch 4/1000000
347/348 [==========

Epoch 31/1000000
347/348 [============================>.] - ETA: 0s - loss: 23.1459 - acc: 0.9675
Epoch 00031: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 161ms/step - loss: 23.1185 - acc: 0.9675 - val_loss: 0.2862 - val_acc: 0.8922
Epoch 32/1000000
347/348 [============================>.] - ETA: 0s - loss: 23.1750 - acc: 0.9677
Epoch 00032: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 161ms/step - loss: 23.1348 - acc: 0.9677 - val_loss: 0.4121 - val_acc: 0.8392
Epoch 33/1000000
347/348 [============================>.] - ETA: 0s - loss: 23.0939 - acc: 0.9677
Epoch 00033: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 161ms/step - loss: 23.0690 - acc: 0.9677 - val_loss: 0.9567 - val_acc: 0.6743
Epoch 34/1000000
347/348 [============================>.] - ETA: 0s - loss: 22.9594 - acc: 0.9677
Epoch 00034: val_acc did not improve from 0.91337
348/348 [========================

347/348 [============================>.] - ETA: 0s - loss: 19.8759 - acc: 0.9717
Epoch 00061: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 160ms/step - loss: 19.8542 - acc: 0.9717 - val_loss: 3.4332 - val_acc: 0.5366
Epoch 62/1000000
347/348 [============================>.] - ETA: 0s - loss: 19.8331 - acc: 0.9719
Epoch 00062: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 160ms/step - loss: 19.8053 - acc: 0.9719 - val_loss: 1.8050 - val_acc: 0.6448
Epoch 63/1000000
347/348 [============================>.] - ETA: 0s - loss: 19.7088 - acc: 0.9720
Epoch 00063: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 160ms/step - loss: 19.7122 - acc: 0.9720 - val_loss: 0.4186 - val_acc: 0.8406
Epoch 64/1000000
347/348 [============================>.] - ETA: 0s - loss: 19.6710 - acc: 0.9720
Epoch 00064: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 160

Epoch 121/1000000
347/348 [============================>.] - ETA: 0s - loss: 16.4559 - acc: 0.9761
Epoch 00121: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 160ms/step - loss: 16.4398 - acc: 0.9761 - val_loss: 0.4774 - val_acc: 0.8386
Epoch 122/1000000
347/348 [============================>.] - ETA: 0s - loss: 16.3568 - acc: 0.9762
Epoch 00122: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 160ms/step - loss: 16.3322 - acc: 0.9762 - val_loss: 0.4213 - val_acc: 0.8540
Epoch 123/1000000
347/348 [============================>.] - ETA: 0s - loss: 16.3461 - acc: 0.9762
Epoch 00123: val_acc did not improve from 0.91337
348/348 [==============================] - 56s 161ms/step - loss: 16.3262 - acc: 0.9762 - val_loss: 0.3990 - val_acc: 0.8751
Epoch 124/1000000
347/348 [============================>.] - ETA: 0s - loss: 16.5453 - acc: 0.9760
Epoch 00124: val_acc did not improve from 0.91337
348/348 [====================

KeyboardInterrupt: 

In [8]:
from sklearn.metrics import accuracy_score
pred = tf.keras.utils.to_categorical(np.argmax(model.predict(x=test_x, batch_size=1), axis=-1), num_classes=4)
accuracy_score(np.argmax(test_y, -1).ravel(), np.argmax(pred, -1).ravel())

/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


0.9225624682856541